In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"
# # !pip install aiosmtpd
# # !uv pip install aiosmtpd

In [ ]:
# stdlib
import os

environment = os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE", "python")
environment

In [ ]:
# third party
from helpers import SENDER
from helpers import create_user
from helpers import make_user
from helpers import save_users

# syft absolute
import syft as sy

In [ ]:
# third party
# run email server
from helpers import EmailServer
from helpers import SMTPTestServer

email_server = EmailServer()
smtp_server = SMTPTestServer(email_server)

In [ ]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [ ]:
ADMIN_EMAIL, ADMIN_PW = "admin2@bigquery.org", "bqpw2"
high_client = sy.login(
    url="http://localhost:8080", email=ADMIN_EMAIL, password=ADMIN_PW
)

In [ ]:
high_client.users

In [ ]:
# TODO: this should show SyftSuccess?
high_client.api.services.settings.enable_notifications(
    email_sender=SENDER,
    email_server="localhost",
    email_port="1025",
)

In [ ]:
# # TODO: this should show SyftSuccess?
# high_client.api.services.settings.disable_notifications()

In [ ]:
num_users = 5
users = []
email_disable_index = 0
reset_password_index = 1
for i in range(num_users):
    user = make_user()
    user._email_server = email_server
    create_user(high_client, user)
    user.client = high_client
    if email_disable_index == i:
        user.email_disabled = True
    if reset_password_index == i:
        user.reset_password = True
    users.append(user)

In [ ]:
save_users(users)

In [ ]:
# everyone gets a welcome email
server_name = high_client.name
for user in users:
    emails = user.emails
    assert len(emails) == 1
    welcome_email = user.emails[0]
    assert welcome_email.email_from == SENDER
    assert len(welcome_email.email_to) == 1
    assert welcome_email.email_to[0] == user.email
    assert f"Welcome to {server_name}" in welcome_email.email_content

In [ ]:
# one user disables notifications
# one user requests a password reset
no_email_user = None
reset_password_user = None
for user in users:
    user.client = high_client  # get user client
    if user.email_disabled:
        no_email_user = user
        # disable for this user only
        user.client.api.notifications.deactivate()

    if user.reset_password:
        # ask admin for forgot password flow
        user.client.guest().forgot_password(email=user.email)
        assert "Password Reset Requested" in user.emails[1].email_content
        reset_password_user = user

In [ ]:
ds0 = users[0]

In [ ]:
ds0_user = ds0.client.account
ds0_user

In [ ]:
# with sy.raises(
# sy.SyftException(public_message="*tried to update user*"
# ), show=True): this is different on k8s no idea why
with sy.raises(sy.SyftException, show=True):
    ds0.client.users.update(uid=ds0_user.id, role="admin")

In [ ]:
# with sy.raises(sy.SyftException(public_message="*tried to update user*"), show=True):
with sy.raises(sy.SyftException, show=True):
    ds0_user.update(role="admin")

In [ ]:
# TODO: test disabling and re-enabling all notifications

In [ ]:
# high_client.api.services.settings.disable_notifications()
# high_client.api.services.settings.enable_notifications()

In [ ]:
token = reset_password_user.get_token()
token

In [ ]:
passw = reset_password_user.make_new_password()
passw

In [ ]:
assert token
assert passw

In [ ]:
output = reset_password_user.client.guest().reset_password(
    token=reset_password_user.reset_token, new_password=reset_password_user.new_password
)
output

In [ ]:
assert isinstance(output, sy.SyftSuccess)

In [ ]:
# relogin
reset_password_user.client = reset_password_user.client

In [ ]:
save_users(users)

In [ ]:
smtp_server.stop()

In [ ]:
server.land()